In [ ]:
# generate REP database 


In [2]:
# Define the string to be written
content = "Hello, this is a text file written in Julia."

# Open the file and write to it
open("output.txt", "w") do file
    write(file, content)
end


44

In [24]:
# returns the nth letter of the infinite alphabet 
# a b c ... x y z aa ab ac ... ax ay az ba bb bc ... bx by bz ... ca cb cd ....

function nth_letter(n::Int)
    # Define the alphabet
    alphabet = "abcdefghijklmnopqrstuvwxyz"
    
    # Initialize the result
    result = ""

    # Compute the letters iteratively for n > 26
    while n > 0
        n -= 1  # Adjust for one-based index
        result = string(alphabet[n % 26 + 1], result)
        n = div(n, 26)
    end

    return result
end


function generate_srctarstr_pair(str_length::Int)
    src = Vector{String}() 
    tar = Vector{String}() 

    alphabet = Set{String}() 
    
    # TARGET 
    # select (w repetiion) |S| numbers from the set {1, .., |S|}
    for i in 1:str_length 
        gene = nth_letter(rand(1:str_length))
        push!(tar, gene) 
        push!(alphabet, gene)
    end 
    tar = tar[1:end]

    println(tar)
    print("unique genes: ", length(alphabet), "  total genes: ", length(tar))

    # SOURCE 
    # shuffle genes randomly & separate into rand # chroms 
    
    
    
    # convert arrays of arrays to strings 
    target_genome = join(tar, "'")
    # source_genome = join()

    # return src, target_genome
end 

generate_srctarstr_pair(100)

["f", "bz", "bf", "cl", "be", "p", "cr", "bf", "bu", "ba", "x", "au", "k", "ch", "ck", "k", "ay", "ad", "s", "af", "bz", "h", "bv", "al", "bb", "cn", "cj", "cj", "am", "cs", "ar", "at", "j", "ct", "r", "ca", "l", "bg", "y", "bv", "h", "bg", "by", "cf", "bv", "ax", "ab", "aa", "ah", "bi", "as", "bd", "au", "z", "aw", "an", "bj", "b", "y", "an", "cu", "cn", "u", "ab", "ce", "am", "q", "cu", "ad", "bc", "b", "bb", "ax", "q", "av", "cb", "h", "as", "n", "ct", "cd", "aw", "ab", "bg", "aa", "as", "cd", "bx", "at", "cc", "ba", "ca", "ah", "ad", "l", "n", "u", "af", "ba", "ce"]
unique genes: 60  total genes: 100

"f'bz'bf'cl'be'p'cr'bf'bu'ba'x'au'k'ch'ck'k'ay'ad's'af'bz'h'bv'al'bb'cn'cj'cj'am'cs'ar'at'j'ct'r'ca'l'bg'y'bv'h'bg'by'cf'bv'ax'ab'aa'ah'bi'as'bd'au'z'aw'an'bj'b'y'an'cu'cn'u'ab'ce'am'q'cu'ad'bc'b'bb'ax'q'av'cb'h'as'n'ct'cd'aw'ab'bg'aa'as'cd'bx'at'cc'ba'ca'ah'ad'l'n'u'af'ba'ce"